In [14]:
import pandas as pd
import os

# Налаштування папок
INPUT_FOLDER = "filtered_data"
RESULTS_FOLDER = "results"
os.makedirs(RESULTS_FOLDER, exist_ok=True)

# Визначені типи закладів
selected_types = ["ліцей", "гімназія", "навчально-виховний комплекс", 
                  "спеціалізована школа", "заклад фахової передвищої освіти", "середня загальноосвітня школа"]

def count_education_types_by_location(df, year):
    # Перевіряємо наявність потрібних колонок
    required_cols = ['education_org_type', 'territory_type']
    if not all(col in df.columns for col in required_cols):
        print(f"⚠️ Відсутні колонки: education_org_type або territory_type у файлі для {year}")
        return None
    
    # Фільтруємо лише вибрані типи
    df_filtered = df[df['education_org_type'].isin(selected_types)].copy()
    
    # Отримуємо унікальні типи поселень для цього року
    valid_locations = df_filtered['territory_type'].dropna().unique().tolist()
    if not valid_locations:
        print(f"⚠️ Жодних даних про territory_type для {year}")
        return None
    
    # Підраховуємо кількість для кожного типу закладу за типом поселення
    result = df_filtered.groupby(['territory_type', 'education_org_type']).size().unstack(fill_value=0)
    
    # Фільтруємо лише задані типи поселень, що є в даних
    result = result.loc[valid_locations]
    
    return result

# Обробка файлів
all_results = {}
for filename in os.listdir(INPUT_FOLDER):
    if not filename.endswith(".csv"):
        continue
    
    file_path = os.path.join(INPUT_FOLDER, filename)
    year = filename.split('.')[0]  # Отримуємо рік із назви файлу
    print(f"🔄 Обробка {filename} ({year})...")
    
    try:
        df = pd.read_csv(file_path, encoding='utf-8', on_bad_lines='skip', low_memory=False)
        print(f"   Завантажено {len(df)} рядків, {len(df.columns)} стовпців")
        
        result = count_education_types_by_location(df, year)
        if result is not None:
            all_results[year] = result
            print(f"✅ Дані для {filename} ({year}) оброблено")
    
    except Exception as e:
        print(f"❌ Помилка у файлі {filename}: {e}")
        import traceback
        traceback.print_exc()

# Збереження результатів для кожного року
if all_results:
    for year, result in all_results.items():
        output_path = os.path.join(RESULTS_FOLDER, f"education_types_by_location_{year}.csv")
        result.to_csv(output_path)
        print(f"✅ Результати для {year} збережено у: {output_path}")
else:
    print("⚠️ Жодних даних не оброблено")

print(f"\n🎉 Аналіз завершено! Результати збережено в '{RESULTS_FOLDER}'")

🔄 Обробка 2020.csv (2020)...
   Завантажено 201212 рядків, 131 стовпців
✅ Дані для 2020.csv (2020) оброблено
🔄 Обробка 2021.csv (2021)...


KeyboardInterrupt: 

In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Налаштування папок
INPUT_FOLDER = "filtered_data"
RESULTS_FOLDER = "results"
GRAPHS_FOLDER = os.path.join(RESULTS_FOLDER, "graphs")
for folder in [RESULTS_FOLDER, GRAPHS_FOLDER]:
    os.makedirs(folder, exist_ok=True)

# Визначені типи закладів
selected_types = ["ліцей", "гімназія", "навчально-виховний комплекс", 
                  "спеціалізована школа", "заклад фахової передвищої освіти", "середня загальноосвітня школа"]

def visualize_education_types_by_location_2022():
    # Шлях до файлу 2022 року
    filename = "2022.csv"
    file_path = os.path.join(INPUT_FOLDER, filename)
    
    print(f"🔄 Обробка {filename}...")
    
    try:
        # Завантажуємо файл
        df = pd.read_csv(file_path, encoding='utf-8', on_bad_lines='skip', low_memory=False)
        print(f"   Завантажено {len(df)} рядків, {len(df.columns)} стовпців")
        
        # Перевіряємо наявність потрібних колонок
        required_cols = ['education_org_type', 'territory_type']
        if not all(col in df.columns for col in required_cols):
            print("⚠️ Відсутні колонки: education_org_type або territory_type")
            return
        
        # Фільтруємо лише вибрані типи
        df_filtered = df[df['education_org_type'].isin(selected_types)].copy()
        
        # Отримуємо унікальні типи поселень і фіксуємо порядок
        locations = ["селище, село", "селище міського типу", "місто"]
        df_filtered = df_filtered[df_filtered['territory_type'].isin(locations)]
        
        if df_filtered.empty:
            print("⚠️ Жодних даних про territory_type серед заданих категорій")
            return
        
        # Підраховуємо загальну кількість кожного типу поселення
        total_locations = df_filtered['territory_type'].value_counts().reindex(locations, fill_value=0)
        
        # Підраховуємо кількість для кожного типу закладу за типом поселення
        result = df_filtered.groupby(['territory_type', 'education_org_type']).size().unstack(fill_value=0)
        
        # Додаємо всі selected_types, навіть якщо їх немає в даних
        for edu_type in selected_types:
            if edu_type not in result.columns:
                result[edu_type] = 0
        
        # Фільтруємо лише задані типи поселень у заданому порядку
        result = result.reindex(index=locations)
        
        # Виводимо наявні типи для діагностики
        print(f"   Наявні типи закладів: {list(result.columns)}")
        
        # Обчислюємо відносні значення: кількість закладу / загальну кількість поселення
        relative_result = result.div(total_locations, axis=0)
        
        # Перетворюємо в довгий формат для seaborn
        relative_result_long = relative_result.reset_index().melt(
            id_vars='territory_type',
            var_name='education_org_type',
            value_name='relative_count'
        )
        
        # Налаштування стилю seaborn
        sns.set_style("whitegrid")
        plt.figure(figsize=(10, 6))
        
        # Створюємо stacked bar chart за допомогою seaborn
        sns.catplot(
            x='territory_type',
            y='relative_count',
            hue='education_org_type',
            data=relative_result_long,
            kind='bar',
            height=6,
            aspect=1.5,
            palette='Set3',
            alpha=0.8
        )
        
        # Додавання заголовків і міток
        plt.title("Відносний розподіл типів закладів за типами поселень (2022 рік)")
        plt.xlabel("Тип поселення")
        plt.ylabel("Відносна кількість закладів на поселення")
        
        # Налаштування осі X
        plt.xticks(rotation=45, ha='right')
        
        # Збереження графіка
        graph_path = os.path.join(GRAPHS_FOLDER, "education_types_by_location_2022_relative.png")
        plt.tight_layout()
        plt.savefig(graph_path, dpi=300, bbox_inches="tight")
        plt.close()
        print(f"✅ Графік збережено у: {graph_path}")
    
    except Exception as e:
        print(f"❌ Помилка у файлі {filename}: {e}")
        import traceback
        traceback.print_exc()

# Виклик функції
visualize_education_types_by_location_2022()

print(f"\n🎉 Аналіз завершено! Результати збережено в '{RESULTS_FOLDER}'")

🔄 Обробка 2022.csv...
   Завантажено 213647 рядків, 36 стовпців
   Наявні типи закладів: ['гімназія', 'заклад фахової передвищої освіти', 'ліцей', 'навчально-виховний комплекс', 'середня загальноосвітня школа', 'спеціалізована школа']
✅ Графік збережено у: results/graphs/education_types_by_location_2022_relative.png

🎉 Аналіз завершено! Результати збережено в 'results'


<Figure size 1000x600 with 0 Axes>